# Black Scholes Multi Stock Option Pricing & Greeks

In [31]:
# Step 1 is to download the entire relevant libraries at one place

import pandas as pd
import yfinance as yf
import numpy as np
import matplotlib.pyplot as plt
from datetime import datetime, timezone
from scipy import stats # used for BS cdf


# Step 2 is to download the relevant data

tickers = ["AAPL", "MSFT", "TSLA", "AMZN", "NVDA"]

raw_data_stocks = yf.download(tickers, period = '1y', auto_adjust = False) # this contains all the relavant stock data. 

# download options data as a nested dictionary

options_data = {}

for tk in tickers:
    stock = yf.Ticker(tk)
    ticker_dict = {}
    
    for exp in stock.options:  # loop over all expiries
        chain = stock.option_chain(exp) # .option_chain(exp) → method that requests the full option chain for that ticker at a specific expiration date
        ticker_dict[exp] = {
            "calls": chain.calls, # DataFrame of all the calls 
            "puts": chain.puts   # DataFrame of all the puts
        }
    
    options_data[tk] = ticker_dict # ticker_dict = all option data for one ticker, organised by expiry.
                                    #options_data = all option data for all tickers, each ticker has its own ticker_dict.

#print(raw_data_stocks['Adj Close'])

# This is the preparation of the stock inputs data. 

df_returns = raw_data_stocks['Adj Close'].pct_change().dropna()
df_returns = df_returns[1:]
sigma_series = df_returns.std()*np.sqrt(252)

S0_series = raw_data_stocks['Adj Close'].iloc[-1]
# use below S0 for current prices 
#S0_series = pd.Series({tk: yf.Ticker(tk).info['currentPrice'] for tk in tickers})
# S0_series = pd.Series({
#     tk: yf.Ticker(tk).history(period="1d", interval="1m")["Close"].iloc[-1]
#     for tk in tickers
# })


stock_summary_df = pd.DataFrame( { 
    'S0' : S0_series , 
    'sigma' : sigma_series 
} )

# This is the preparation of options DataFrame

frames = []
for tk in tickers:
    for exp, opt_dict in options_data[tk].items():
        for option_type, df in [("call", opt_dict["calls"]), ("put", opt_dict["puts"])]:
            temp = df.copy()
            temp["ticker"] = tk
            temp["expiry"] = exp
            temp["option_type"] = option_type
            frames.append(temp)

options_df = pd.concat(frames, ignore_index=True)

rows = []
today = datetime.now(timezone.utc).replace(tzinfo=None)


for tk in tickers:
    # --- Stock inputs ---
    S0 = stock_summary_df.loc[tk, "S0"]
    sigma = stock_summary_df.loc[tk, "sigma"]

    # --- Expiry selection ---
    expiries = pd.to_datetime(options_df[options_df["ticker"] == tk]["expiry"].unique())
    valid_exps = [exp for exp in expiries if (exp - today).days > 1]
    chosen_exp = min(valid_exps, key=lambda d: abs((d - today).days - 30))  # nearest to 30 days
    T = (chosen_exp - today).days / 365.0

    # --- ATM strike selection ---
    subset = options_df[(options_df["ticker"] == tk) & (options_df["expiry"] == str(chosen_exp.date()))]
    atm_strike = subset.iloc[(subset["strike"] - S0).abs().argsort()[:1]]["strike"].values[0]

    # --- Loop over call & put ---
    for option_type in ["call", "put"]:
        contract = subset[(subset["strike"] == atm_strike) & (subset["option_type"] == option_type)]
        marketPrice = contract["ask"].values[0]

        rows.append({
            "ticker": tk,
            "expiry": chosen_exp,
            "S0": S0,
            "sigma": sigma,
            "T": T,
            "strike": atm_strike,
            "rf": 0.047,
            "option_type": option_type,
            "marketPrice": marketPrice
        })

# Build DataFrame
model_inputs_df = pd.DataFrame(rows)

df_in = model_inputs_df.copy()

# --- Black–Scholes closed form ---
d1 = (np.log(df_in["S0"]/df_in["strike"]) +
      (df_in["rf"] + 0.5*df_in["sigma"]**2) * df_in["T"]) / (df_in["sigma"]*np.sqrt(df_in["T"]))
d2 = d1 - df_in["sigma"]*np.sqrt(df_in["T"])

df_in["bs_price"] = np.where(
    df_in["option_type"]=="call",
    df_in["S0"]*stats.norm.cdf(d1) - df_in["strike"]*np.exp(-df_in["rf"]*df_in["T"])*stats.norm.cdf(d2),
    df_in["strike"]*np.exp(-df_in["rf"]*df_in["T"])*stats.norm.cdf(-d2) - df_in["S0"]*stats.norm.cdf(-d1)
)

# --- Monte Carlo pricing ---
class OptionPricing:
    def __init__(self, S0, E, rf, T, sigma, iterations, option_type='call'):
        self.S0 = S0
        self.E = E
        self.rf = rf
        self.T = T
        self.sigma = sigma 
        self.iterations = iterations 
        self.option_type = option_type

    def mc_option_pricing(self):
        rand = np.random.normal(0, 1, self.iterations)
        S = self.S0*np.exp((self.rf - 0.5*self.sigma**2)*self.T + self.sigma*rand*np.sqrt(self.T))
        if self.option_type == 'call':
            payoff = np.maximum(S - self.E, 0)
        else:
            payoff = np.maximum(self.E - S, 0)
        return np.exp(-self.rf*self.T) * np.mean(payoff)

for i, row in df_in.iterrows():
    pricer = OptionPricing(row["S0"], row["strike"], row["rf"], row["T"], row["sigma"], 10000, row["option_type"])
    df_in.loc[i, "mc_price"] = pricer.mc_option_pricing()

# Copy inputs + prices
df_out = df_in.copy()

# Mispricing
df_out["mispricing_abs"] = df_out["bs_price"] - df_out["marketPrice"]
df_out["mispricing_pct"] = (df_out["mispricing_abs"] / df_out["marketPrice"]) * 100

# Greeks
pdf_d1 = stats.norm.pdf(d1)

df_out["delta"] = np.where(df_out["option_type"]=="call", stats.norm.cdf(d1), stats.norm.cdf(d1)-1)
df_out["gamma"] = pdf_d1 / (df_out["S0"]*df_out["sigma"]*np.sqrt(df_out["T"]))
df_out["vega"]  = df_out["S0"]*pdf_d1*np.sqrt(df_out["T"])
df_out["theta"] = np.where(
    df_out["option_type"]=="call",
    -(df_out["S0"]*pdf_d1*df_out["sigma"])/(2*np.sqrt(df_out["T"])) - df_out["rf"]*df_out["strike"]*np.exp(-df_out["rf"]*df_out["T"])*stats.norm.cdf(d2),
    -(df_out["S0"]*pdf_d1*df_out["sigma"])/(2*np.sqrt(df_out["T"])) + df_out["rf"]*df_out["strike"]*np.exp(-df_out["rf"]*df_out["T"])*stats.norm.cdf(-d2)
)
df_out["rho"]   = np.where(
    df_out["option_type"]=="call",
    df_out["strike"]*df_out["T"]*np.exp(-df_out["rf"]*df_out["T"])*stats.norm.cdf(d2),
    -df_out["strike"]*df_out["T"]*np.exp(-df_out["rf"]*df_out["T"])*stats.norm.cdf(-d2)
)

trader_view = df_out[[
    "ticker", "S0", "expiry", "strike", "option_type",
    "marketPrice", "bs_price", "mc_price",
    "mispricing_abs", "mispricing_pct",
    "delta", "gamma", "vega", "theta"
]]

#print(trader_view.round(3))

from IPython.display import display

styled = trader_view.round(3).style.format({
    "marketPrice": "{:.2f}",
    "S0" : "{:0.2f}",
    "bs_price": "{:.2f}",
    "strike": "{:.2f}",
    "mc_price": "{:.2f}",
    "mispricing_abs": "{:+.2f}",
    "mispricing_pct": "{:+.1f}%",
    "delta": "{:.3f}",
    "gamma": "{:.3f}",
    "vega": "{:.2f}",
    "theta": "{:.2f}"
}).background_gradient(
    subset=["mispricing_pct"], cmap="RdYlGn"
).set_table_styles(
    [{'selector': 'th', 'props': [('font-weight', 'bold'), ('text-align', 'center')]}]
)

display(styled)

[*********************100%***********************]  5 of 5 completed


,ticker,S0,expiry,strike,option_type,marketPrice,bs_price,mc_price,mispricing_abs,mispricing_pct,delta,gamma,vega,theta
0,AAPL,255.46,2025-10-31 00:00:00,255.00,call,9.00,10.77,10.84,+1.77,+19.7%,0.544,0.016,30.46,-61.08
1,AAPL,255.46,2025-10-31 00:00:00,255.00,put,7.60,9.23,9.09,+1.63,+21.5%,-0.456,0.016,30.46,-49.15
2,MSFT,511.46,2025-10-31 00:00:00,510.00,call,18.40,17.06,17.13,-1.34,-7.3%,0.553,0.010,60.82,-96.04
3,MSFT,511.46,2025-10-31 00:00:00,510.00,put,14.80,13.44,13.36,-1.36,-9.2%,-0.447,0.010,60.82,-72.17
4,TSLA,440.40,2025-10-31 00:00:00,440.00,call,34.40,38.62,39.47,+4.22,+12.3%,0.552,0.004,52.38,-216.00
5,TSLA,440.40,2025-10-31 00:00:00,440.00,put,32.25,36.35,36.23,+4.10,+12.7%,-0.448,0.004,52.38,-195.41
6,AMZN,219.78,2025-10-31 00:00:00,220.00,call,10.20,9.26,9.00,-0.94,-9.3%,0.533,0.018,26.27,-54.19
7,AMZN,219.78,2025-10-31 00:00:00,220.00,put,9.40,8.54,8.51,-0.86,-9.1%,-0.467,0.018,26.27,-43.90
8,NVDA,178.19,2025-10-31 00:00:00,180.00,call,7.25,10.16,10.00,+2.91,+40.1%,0.514,0.015,21.36,-62.69
9,NVDA,178.19,2025-10-31 00:00:00,180.00,put,8.35,11.20,10.96,+2.85,+34.2%,-0.486,0.015,21.36,-54.27
